# Implémentation un moteur d’inférence API . 
L'objectif de l’API est de renvoyer le sentiment à réception d’un texte brut 'API .

In [1]:
import os
import re
import string
import joblib
import pandas as pd
import flask
from flask import Flask, jsonify, request
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer, text_to_word_sequence

In [7]:
# initialize our Flask application and the Keras model
app = flask.Flask(__name__)

# Load data
df = pd.read_csv('input_app/df_cleaned_docs.csv',encoding='ISO-8859-1')


<bound method NDFrame.head of            target                                               text   
0        NEGATIVE  @switchfoot http://twitpic.com/2y1zl - Awww, t...  \
1        NEGATIVE  is upset that he can't update his Facebook by ...   
2        NEGATIVE  @Kenichan I dived many times for the ball. Man...   
3        NEGATIVE    my whole body feels itchy and like its on fire    
4        NEGATIVE  @nationwideclass no, it's not behaving at all....   
...           ...                                                ...   
1597484  POSITIVE  Just woke up. Having no school is the best fee...   
1597485  POSITIVE  TheWDB.com - Very cool to hear old Walt interv...   
1597486  POSITIVE  Are you ready for your MoJo Makeover? Ask me f...   
1597487  POSITIVE  Happy 38th Birthday to my boo of alll time!!! ...   
1597488  POSITIVE  happy #charitytuesday @theNSPCC @SparksCharity...   

                                                clean_text  
0        switchfoot httptwitpiccom awww s bu

In [ ]:
# tokenizer
tokenizer = Tokenizer(nb_words=14225)
tokenizer.fit_on_texts( df['clean_text'])

# Load model
model = keras.models.load_model("input_app/word2vec_lstm_epoch_30.h5")

C:\Users\doly9\AppData\Roaming\Python\Python311\site-packages\keras\preprocessing\text.py:246: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn(


In [ ]:
# remove special characters
def remove_special_characters(text):
    #Removing numerical values, Removing Digits and words containing digits
    text= re.sub('\w*\d\w*','', text)
    #Removing punctations
    text= re.sub('[%s]' % re.escape(string.punctuation), '', text)
    #Removing Extra Spaces
    text =re.sub(' +', ' ',text)
    # remove stock market tickers like $GE
    text = re.sub(r'\$\w*', '',text)
    # remove old style retweet text "RT"
    text = re.sub(r'^RT[\s]+', '',text)
    # remove hyperlinks
    text = re.sub(r'https?:\/\/.*[\r\n]*', '',text)
    # remove hashtags
    # only removing the hash # sign from the word
    text = re.sub(r'#', '',text)
    text = text.lower()

    return text

def tokenize_stopwords_lemmatize(texts, allowed_postags=['NOUN','ADJ','ADV']):
    tokenized_docs = texts.apply(lambda x: ' '.join([token.lemma_.lower() for token in list(nlp(x)) if token.is_alpha and not token.is_stop]))
    return tokenized_docs

def encode_text(text):
    #tokenizer = Tokenizer(nb_words=14225)
    #tokenizer.fit_on_texts([text])
    #sequence = tokenizer.texts_to_sequences([text])[0]
    sequence = tokenizer.texts_to_sequences([text])
    padded_test_sequence = pad_sequences(sequence, maxlen=30)
    print(padded_test_sequence)
    return padded_test_sequence

In [ ]:
@app.route('/', methods=['GET'])
def ping():
    return 'Hello, World!'


In [ ]:
@app.route('/predict', methods=['POST'])

def predict():
    #global model
    data = request.get_json(force=True)
    text = data['text']
    #print(text)
    cleaned_text = remove_special_characters(text)
    encoded_text = encode_text(cleaned_text)
    predict_output = model.predict(encoded_text)[0][0]
    
    if predict_output>0.5:
        output = {'result of model': "The sentiment of this text is positive, with a polarity score of {0:.2f}".format(predict_output)}
    else:
        output = {'result of model': "The sentiment of this text is negative, with a polarity score of {0:.2f}".format(predict_output)}

    
    return jsonify(output)

In [ ]:
if __name__ == '__main__':
    #load_model()
    app.run(debug=True,use_reloader=False, port=8000)